***PRODUCER START***

In [1]:
import psycopg2
from confluent_kafka import Producer
import json
from kafka import KafkaProducer
import uuid

***PREPARING TO SEND TO KAFKA***

In [2]:
def dict_serializer(data):
  return json.dumps(data).encode('utf-8')

def sendToKafka(message):
    bootstrap_servers = ['localhost:9092']

    producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=dict_serializer)
    topic = 'temp_05'

    producer.send(topic, message)

    producer.flush()
    producer.close()

***PREPARING DATABASE EXTRACTION***

In [3]:
server = "localhost"
database = "db_02"
username = "postgres"
password = "ABcd@12#$"
source = "creditcard_test"

try:
  conn = psycopg2.connect(dbname=database, user=username, password=password, host=server, port="5432", sslmode='disable')
except psycopg2.Error as e:
  print("Error connecting to PostgreSQL database:", e)
else:
  print("Connection established successfully!")

cursor = conn.cursor()

query =f"SELECT * FROM {source}"
cursor.execute(query)

Connection established successfully!


***SENDING TO KAFKA***

In [4]:
def manage(row):
    data={}

    data['Time']= row[1]
    for i in range (2, 30):
        data[f"V{i-1}"]= float(row[i-1])
    data['Amount']= row[30]
    data['ID'] = row[0]

    sendToKafka(data)

***DATABASE EXTRACTION***

In [5]:
for row in cursor.fetchall():
    manage(row)
    # if(row[1]==154493):
    #     manage(row)

***PRODUCER STOP***

In [ ]:
print('Sent All :)')
cursor.close()
conn.close()

Sent All :)
